In [130]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [131]:
import pickle
with open('ten_raw_data.pickle','rb') as handle:
    raw_data = pickle.load(handle)
#삼성전자, SK하이닉스, 현대차, PSOCO, LG화학, NAVER, 삼성물산
#SK텔레콤, 현대모비스, KB금융
#"005930","000660","005380","047050","051910","035420","028260",
#"017670","012330","105560"

In [132]:
raw_data['005930'].head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,16060.0,16180.0,16000.0,16180.0,5.953252,11950800
2010-01-05,16520.0,16580.0,16300.0,16440.0,6.048919,27925850
2010-01-06,16580.0,16820.0,16520.0,16820.0,6.188735,22948850
2010-01-07,16820.0,16820.0,16260.0,16260.0,5.982689,22107950
2010-01-08,16400.0,16420.0,16120.0,16420.0,6.041558,14777550


In [133]:
raw_data['005930'].tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-07-25,46250.0,46550.0,45900.0,46150.0,46150.0,7222471
2018-07-26,46100.0,47000.0,46000.0,46900.0,46900.0,7374946
2018-07-27,46450.0,47000.0,46450.0,46900.0,46900.0,4762460
2018-07-30,46550.0,46800.0,46350.0,46500.0,46500.0,5723035
2018-07-31,46200.0,46450.0,46000.0,46250.0,46250.0,8033607


In [134]:
len(raw_data)

10

In [135]:
stock_name=['삼성전자', 'SK하이닉스', '현대차', 'PSOCO', 'LG화학', 'NAVER',
'삼성물산','SK텔레콤', '현대모비스', 'KB금융']
stock_num=["005930","000660","005380","047050","051910","035420","028260",
"017670","012330","105560"]
for i in range(10):
    raw_data[stock_name[i]] = raw_data.pop(stock_num[i])

In [136]:
raw_data['삼성전자'].head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,16060.0,16180.0,16000.0,16180.0,5.953252,11950800
2010-01-05,16520.0,16580.0,16300.0,16440.0,6.048919,27925850
2010-01-06,16580.0,16820.0,16520.0,16820.0,6.188735,22948850
2010-01-07,16820.0,16820.0,16260.0,16260.0,5.982689,22107950
2010-01-08,16400.0,16420.0,16120.0,16420.0,6.041558,14777550


In [137]:
raw_data['삼성전자'].Close.pct_change().head(4)

Date
2010-01-04         NaN
2010-01-05    0.016069
2010-01-06    0.023114
2010-01-07   -0.033294
Name: Close, dtype: float64

In [138]:
raw_data['삼성전자'].Close.pct_change().shift(5).head(9)

Date
2010-01-04         NaN
2010-01-05         NaN
2010-01-06         NaN
2010-01-07         NaN
2010-01-08         NaN
2010-01-11         NaN
2010-01-12    0.016069
2010-01-13    0.023114
2010-01-14   -0.033294
Name: Close, dtype: float64

In [139]:
for i in raw_data.keys():
    """
    X
    """
    raw_data[i] = raw_data[i].assign(sh1_pct_change = raw_data[i].Close.pct_change().shift(1))
    raw_data[i] = raw_data[i].assign(sh5_pct_change = raw_data[i].Close.pct_change().shift(5))
    raw_data[i] = raw_data[i].assign(pct_change = raw_data[i].Close.pct_change().fillna(0))
    
    raw_data[i] = raw_data[i].assign(sh1_close_ma_3 = raw_data[i].Close.rolling(window =3).mean().shift(1))
    raw_data[i] = raw_data[i].assign(sh5_close_ma_3 = raw_data[i].Close.rolling(window =3).mean().shift(5))

    raw_data[i] = raw_data[i].assign(sh1_his_vol_7 = raw_data[i]["pct_change"].rolling(window = 7).std().shift(1)*(252**0.5))
    raw_data[i] = raw_data[i].assign(sh5_his_vol_7 = raw_data[i]["pct_change"].rolling(window = 7).std().shift(5)*(252**0.5))
    
    raw_data[i] = raw_data[i].assign(sh1_his_vol_21 = raw_data[i]["pct_change"].rolling(window = 21).std().shift(1)*(252**0.5))
    raw_data[i] = raw_data[i].assign(sh5_his_vol_21 = raw_data[i]["pct_change"].rolling(window = 21).std().shift(5)*(252**0.5))
   
    raw_data[i] = raw_data[i].assign(sh1_volume = raw_data[i].Volume.shift(1))
    raw_data[i] = raw_data[i].assign(sh5_volume = raw_data[i].Volume.shift(5))
    
    """
    Y
    """
    raw_data[i] = raw_data[i].assign(Y1 = raw_data[i].Close - raw_data[i].Close.rolling(window =7).mean().shift(1))
    raw_data[i] = raw_data[i].assign(Y1 = np.where(raw_data[i].Y1>=0, 1, 0))
    raw_data[i] = raw_data[i].assign(Y3 = raw_data[i].Close.rolling(window=7).mean() - raw_data[i].Close.rolling(window =7).mean().shift(1))
    raw_data[i] = raw_data[i].assign(Y3 = np.where(raw_data[i].Y3>=0, 1, 0))
    
    """
    Drop
    """
    raw_data[i] = raw_data[i].dropna(axis=0)

In [140]:
data = {}
for i in raw_data.keys():
    data[i] = raw_data[i].loc['2015-08-01':'2018-07-31']

In [141]:
for i in data.keys():
    if (len(data[i]) < 504):
        del data[i]
    elif (data[i][['Close','Volume','Open']].duplicated().sum() > 20):
        del data[i]

In [142]:
len(data)

10

In [144]:
train1 = {}
train1_idx = {}
test1 = {}
test1_idx = {}

train5 = {}
train5_idx = {}
test5 = {}
test5_idx = {}

for code, df in data.items():
    train1[code] = df.iloc[:int(len(df)*0.8)][["sh1_pct_change","sh1_close_ma_3","sh1_volume","sh1_his_vol_7","sh1_his_vol_21"]]
    train1_idx[code] = df.iloc[:int(len(df)*0.8)]["Y3"]    
    test1[code] = df.iloc[int(len(df)*0.8):][["sh1_pct_change","sh1_close_ma_3","sh1_volume","sh1_his_vol_7","sh1_his_vol_21"]]
    test1_idx[code] = df.iloc[int(len(df)*0.8):]["Y3"]
    
    train5[code] = df.iloc[:int(len(df)*0.8)][["sh5_pct_change","sh5_close_ma_3","sh5_volume","sh5_his_vol_7","sh5_his_vol_21"]]
    train5_idx[code] = df.iloc[:int(len(df)*0.8)]["Y3"]    
    test5[code] = df.iloc[int(len(df)*0.8):][["sh5_pct_change","sh5_close_ma_3","sh5_volume","sh5_his_vol_7","sh5_his_vol_21"]]
    test5_idx[code] = df.iloc[int(len(df)*0.8):]["Y3"]

In [145]:
train1_scale = {}
train1_idx_scale = {}
test1_scale = {}
test1_idx_scale = {}

train5_scale = {}
train5_idx_scale = {}
test5_scale = {}
test5_idx_scale = {}

for code, df in data.items():
    train1_scale[code] = df.iloc[:int(len(df)*0.8)][["sh1_pct_change","sh1_close_ma_3","sh1_volume","sh1_his_vol_7","sh1_his_vol_21"]]
    train1_idx_scale[code] = df.iloc[:int(len(df)*0.8)]["Y3"]    
    test1_scale[code] = df.iloc[int(len(df)*0.8):][["sh1_pct_change","sh1_close_ma_3","sh1_volume","sh1_his_vol_7","sh1_his_vol_21"]]
    test1_idx_scale[code] = df.iloc[int(len(df)*0.8):]["Y3"]
    
    train5_scale[code] = df.iloc[:int(len(df)*0.8)][["sh5_pct_change","sh5_close_ma_3","sh5_volume","sh5_his_vol_7","sh5_his_vol_21"]]
    train5_idx_scale[code] = df.iloc[:int(len(df)*0.8)]["Y3"]    
    test5_scale[code] = df.iloc[int(len(df)*0.8):][["sh5_pct_change","sh5_close_ma_3","sh5_volume","sh5_his_vol_7","sh5_his_vol_21"]]
    test5_idx_scale[code] = df.iloc[int(len(df)*0.8):]["Y3"]

In [146]:
train1['삼성전자'].head()

,sh1_pct_change,sh1_close_ma_3,sh1_volume,sh1_his_vol_7,sh1_his_vol_21
Date,,,,,
2015-08-03,-0.024691,24420.000000,19579450.0,0.330462,0.322405
2015-08-04,-0.008439,23833.333333,9844550.0,0.326568,0.314871
2015-08-05,0.005957,23613.333333,12844350.0,0.335496,0.301013
2015-08-06,-0.019459,23440.000000,12802500.0,0.341812,0.304783
2015-08-07,-0.037964,23040.000000,20199050.0,0.377340,0.327678


In [147]:
li1 = ["sh1_pct_change","sh1_close_ma_3","sh1_volume","sh1_his_vol_7","sh1_his_vol_21"]
li5 = ["sh5_pct_change","sh5_close_ma_3","sh5_volume","sh5_his_vol_7","sh5_his_vol_21"]

In [148]:
train_li = np.array([57,128,15,66,11,223,55,98,664,253], dtype=np.float)
test_li = np.array([227,127,68], dtype=np.float)

std_scaler = StandardScaler()
std_scaler.fit(train_li.reshape(-1,1))

StandardScaler(copy=True, with_mean=True, with_std=True)

In [149]:
std_scaler.transform(train_li.reshape(-1,1))

array([[-0.53829765],
       [-0.15610632],
       [-0.76438267],
       [-0.48985086],
       [-0.78591457],
       [ 0.35527645],
       [-0.54906361],
       [-0.31759562],
       [ 2.7291691 ],
       [ 0.51676575]])

In [150]:
std_scaler.transform(test_li.reshape(-1,1))
# Train set의 mean, std로 정규화를 시켜줘야 한다.

array([[ 0.37680836],
       [-0.1614893 ],
       [-0.47908491]])

In [151]:
std_scaler2 = StandardScaler()
std_scaler2.fit_transform(test_li.reshape(-1,1))
# Test set의 mean, std로 정규화를 시키면 위와 전혀다른 input을 넣게된다.

array([[ 1.31551764],
       [-0.20824797],
       [-1.10726967]])

In [152]:
from sklearn.preprocessing import StandardScaler

for code in data.keys():
    # Create StandardScaler object
    std_scaler1 = StandardScaler()
    std_scaler5 = StandardScaler()
    # fit
    std_scaler1.fit(train1_scale[code][li1].values)
    std_scaler5.fit(train5_scale[code][li5].values)
    # transform
    train1_scale[code][li1] = std_scaler1.transform(train1_scale[code][li1].values)
    test1_scale[code][li1] = std_scaler1.transform(test1_scale[code][li1].values)
    
    train5_scale[code][li5] = std_scaler5.transform(train5_scale[code][li5].values)
    test5_scale[code][li5] = std_scaler5.transform(test5_scale[code][li5].values)

In [157]:
std_scaler = StandardScaler()
test1['삼성전자'][li1] = std_scaler.fit_transform(test1['삼성전자'][li1].values)
test1['삼성전자'].head()
# fit_transform을 적용하여 test set의 mean과 std로 정규화를 적용했을 때

,sh1_pct_change,sh1_close_ma_3,sh1_volume,sh1_his_vol_7,sh1_his_vol_21
Date,,,,,
2017-12-22,-2.543971,0.780330,0.218415,0.416899,0.660755
2017-12-26,0.654199,0.533468,-0.323032,0.563198,0.644052
2017-12-27,-1.625364,-0.019503,0.269266,0.946825,0.936332
2017-12-28,1.348883,0.016704,-0.378840,1.655515,1.349283
2018-01-02,1.806610,0.224068,-0.593986,2.390017,0.850949


In [154]:
test1_scale['삼성전자'].head()
# fit 실시 후 transform을 적용하여 train set의 mean과 std로 정규화를 적용했을 때

,sh1_pct_change,sh1_close_ma_3,sh1_volume,sh1_his_vol_7,sh1_his_vol_21
Date,,,,,
2017-12-22,-2.993225,1.507612,0.678656,0.748069,1.080292
2017-12-26,0.618040,1.459368,-0.139840,0.855585,1.070141
2017-12-27,-1.955966,1.351300,0.755527,1.137513,1.247774
2017-12-28,1.402453,1.358376,-0.224203,1.658330,1.498744
2018-01-02,1.919304,1.398902,-0.549435,2.198117,1.195882


In [158]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn import metrics

xgb_clf = XGBClassifier(random_state=42)
log_clf = LogisticRegression()
svm_clf = SVC()
rf_clf = RandomForestClassifier(random_state=42)
gnb_clf = GaussianNB()
knn_clf = KNeighborsClassifier()
dt_clf = DecisionTreeClassifier()

In [159]:
models=[xgb_clf,rf_clf,log_clf,svm_clf,dt_clf,gnb_clf,knn_clf]
model_name=['xgb','rf','log','svm','dt','gnb','knn']

In [161]:
import warnings
warnings.simplefilter('ignore', DeprecationWarning)
result = {}
# conf_mx = {}
for i in raw_data.keys():
    X_train = train1[i].values
    y_train = train1_idx[i].values
    X_test = test1[i].values
    y_test = test1_idx[i].values
    score = {}
#     confusion = {}
    for clf, name in zip(models, model_name):
        model_score = []
#         confusion_mat = []
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        try:
            y_prob = clf.predict_proba(X_test)
        except NotFittedError:
            clf.probability = True
            y_prob = clf.predict_proba(X_test)
        model_score.append(metrics.accuracy_score(y_test, y_pred))
        model_score.append(metrics.precision_score(y_test, y_pred))
        model_score.append(metrics.recall_score(y_test, y_pred))
        model_score.append(metrics.f1_score(y_test, y_pred))
        model_score.append(metrics.roc_auc_score(y_test, y_prob[:,1]))
        confusion_mat.append(confusion_matrix(y_test, y_pred))
        score[name] = model_score
#         confusion[name] = confusion_mat
    result[i] = score
#     conf_mx[i] = confusion

C:\Users\JinMyungHun\Anaconda3\envs\ds\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\JinMyungHun\Anaconda3\envs\ds\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\JinMyungHun\Anaconda3\envs\ds\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\JinMyungHun\Anaconda3\envs\ds\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [162]:
for i in data.keys():
    print(i)
    print(pd.DataFrame(result[i],index=['accuracy', 'precision', 'recall', 'f1', 'auc']).apply(lambda x : round(x,4)*100),"\n")

삼성전자
              dt    gnb    knn    log     rf     svm    xgb
accuracy   59.18  49.66  49.66  59.18  49.66   50.34  54.42
precision  57.95   0.00   0.00  58.97  50.00   50.34  60.00
recall     68.92   0.00   0.00  62.16  74.32  100.00  28.38
f1         62.96   0.00   0.00  60.53  59.78   66.97  38.53
auc        59.12  62.46  50.00  62.37  51.77   50.00  53.47 

SK하이닉스
              dt    gnb    knn     log     rf     svm    xgb
accuracy   48.98  50.34  51.02   53.74  60.54   53.74  59.86
precision  53.03  52.14  53.10   53.74  62.96   53.74  59.09
recall     44.30  92.41  75.95  100.00  64.56  100.00  82.28
f1         48.28  66.67  62.50   69.91  63.75   69.91  68.78
auc        49.36  53.83  45.81   56.57  62.89   50.00  64.16 

현대차
              dt    gnb    knn    log     rf     svm    xgb
accuracy   53.06  53.74  54.42  39.46  52.38   46.94  48.98
precision  50.00  50.54  51.28  36.49  49.28   46.94  46.34
recall     53.62  68.12  57.97  39.13  49.28  100.00  55.07
f1         51.

In [163]:
import warnings
warnings.simplefilter('ignore', DeprecationWarning)
result = {}
# conf_mx = {}
for i in raw_data.keys():
    X_train = train5[i].values
    y_train = train5_idx[i].values
    X_test = test5[i].values
    y_test = test5_idx[i].values
    score = {}
#     confusion = {}
    for clf, name in zip(models, model_name):
        model_score = []
#         confusion_mat = []
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        try:
            y_prob = clf.predict_proba(X_test)
        except NotFittedError:
            clf.probability = True
            y_prob = clf.predict_proba(X_test)
        model_score.append(metrics.accuracy_score(y_test, y_pred))
        model_score.append(metrics.precision_score(y_test, y_pred))
        model_score.append(metrics.recall_score(y_test, y_pred))
        model_score.append(metrics.f1_score(y_test, y_pred))
        model_score.append(metrics.roc_auc_score(y_test, y_prob[:,1]))
        confusion_mat.append(confusion_matrix(y_test, y_pred))
        score[name] = model_score
#         confusion[name] = confusion_mat
    result[i] = score
#     conf_mx[i] = confusion

C:\Users\JinMyungHun\Anaconda3\envs\ds\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\JinMyungHun\Anaconda3\envs\ds\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\JinMyungHun\Anaconda3\envs\ds\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\JinMyungHun\Anaconda3\envs\ds\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [164]:
for i in data.keys():
    print(i)
    print(pd.DataFrame(result[i],index=['accuracy', 'precision', 'recall', 'f1', 'auc']).apply(lambda x : round(x,4)*100),"\n")

삼성전자
              dt    gnb    knn     log     rf     svm    xgb
accuracy   64.63  48.98  46.94   52.38  63.95   50.34  62.59
precision  62.79  49.18  47.96   51.39  64.00   50.34  71.11
recall     72.97  40.54  63.51  100.00  64.86  100.00  43.24
f1         67.50  44.44  54.65   67.89  64.43   66.97  53.78
auc        64.57  59.20  44.08   50.98  67.75   50.00  65.35 

SK하이닉스
              dt    gnb    knn     log     rf     svm    xgb
accuracy   56.46  53.06  52.38   53.74  59.18   53.74  56.46
precision  61.90  53.42  54.05   53.74  70.21   53.74  65.31
recall     49.37  98.73  75.95  100.00  41.77  100.00  40.51
f1         54.93  69.33  63.16   69.91  52.38   69.91  50.00
auc        57.04  59.62  53.27   35.31  63.45   50.00  65.87 

현대차
              dt    gnb    knn    log     rf     svm    xgb
accuracy   53.74  57.82  54.42  46.94  57.82   46.94  64.63
precision  50.59  57.78  51.25  41.82  53.93   46.94  58.25
recall     62.32  37.68  59.42  33.33  69.57  100.00  86.96
f1      

In [165]:
import warnings
warnings.simplefilter('ignore', DeprecationWarning)
result = {}
# conf_mx = {}
for i in raw_data.keys():
    X_train = train1_scale[i].values
    y_train = train1_idx_scale[i].values
    X_test = test1_scale[i].values
    y_test = test1_idx_scale[i].values
    score = {}
#     confusion = {}
    for clf, name in zip(models, model_name):
        model_score = []
#         confusion_mat = []
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        try:
            y_prob = clf.predict_proba(X_test)
        except NotFittedError:
            clf.probability = True
            y_prob = clf.predict_proba(X_test)
        model_score.append(metrics.accuracy_score(y_test, y_pred))
        model_score.append(metrics.precision_score(y_test, y_pred))
        model_score.append(metrics.recall_score(y_test, y_pred))
        model_score.append(metrics.f1_score(y_test, y_pred))
        model_score.append(metrics.roc_auc_score(y_test, y_prob[:,1]))
        confusion_mat.append(confusion_matrix(y_test, y_pred))
        score[name] = model_score
#         confusion[name] = confusion_mat
    result[i] = score
#     conf_mx[i] = confusion

In [166]:
for i in data.keys():
    print(i)
    print(pd.DataFrame(result[i],index=['accuracy', 'precision', 'recall', 'f1', 'auc']).apply(lambda x : round(x,4)*100),"\n")

삼성전자
              dt    gnb    knn    log     rf    svm    xgb
accuracy   55.10  51.70  65.99  51.02  53.74  63.27  54.42
precision  55.26  51.81  63.04  51.32  55.77  59.26  54.55
recall     56.76  58.11  78.38  52.70  39.19  86.49  56.76
f1         56.00  54.78  69.88  52.00  46.03  70.33  55.63
auc        55.09  54.87  68.51  58.05  58.17  63.01  61.64 

SK하이닉스
              dt    gnb    knn    log     rf    svm    xgb
accuracy   48.98  59.18  65.31  61.22  60.54  60.54  59.86
precision  53.03  57.48  72.58  58.46  62.96  59.63  59.09
recall     44.30  92.41  56.96  96.20  64.56  82.28  82.28
f1         48.28  70.87  63.83  72.73  63.75  69.15  68.78
auc        49.36  65.71  69.95  68.54  62.89  68.71  64.16 

현대차
              dt    gnb    knn    log     rf    svm    xgb
accuracy   52.38  58.50  60.54  63.95  52.38  58.50  48.98
precision  49.33  55.88  57.33  61.11  49.28  55.41  46.34
recall     53.62  55.07  62.32  63.77  49.28  59.42  55.07
f1         51.39  55.47  59.72  62.4

In [167]:
import warnings
warnings.simplefilter('ignore', DeprecationWarning)
result = {}
# conf_mx = {}
for i in raw_data.keys():
    X_train = train5_scale[i].values
    y_train = train5_idx_scale[i].values
    X_test = test5_scale[i].values
    y_test = test5_idx_scale[i].values
    score = {}
#     confusion = {}
    for clf, name in zip(models, model_name):
        model_score = []
#         confusion_mat = []
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        try:
            y_prob = clf.predict_proba(X_test)
        except NotFittedError:
            clf.probability = True
            y_prob = clf.predict_proba(X_test)
        model_score.append(metrics.accuracy_score(y_test, y_pred))
        model_score.append(metrics.precision_score(y_test, y_pred))
        model_score.append(metrics.recall_score(y_test, y_pred))
        model_score.append(metrics.f1_score(y_test, y_pred))
        model_score.append(metrics.roc_auc_score(y_test, y_prob[:,1]))
        confusion_mat.append(confusion_matrix(y_test, y_pred))
        score[name] = model_score
#         confusion[name] = confusion_mat
    result[i] = score
#     conf_mx[i] = confusion

In [168]:
for i in data.keys():
    print(i)
    print(pd.DataFrame(result[i],index=['accuracy', 'precision', 'recall', 'f1', 'auc']).apply(lambda x : round(x,4)*100),"\n")

삼성전자
              dt    gnb    knn    log     rf    svm    xgb
accuracy   61.90  60.54  57.82  61.22  63.95  61.22  62.59
precision  60.98  62.90  60.00  63.93  64.00  66.67  71.11
recall     67.57  52.70  48.65  52.70  64.86  45.95  43.24
f1         64.10  57.35  53.73  57.78  64.43  54.40  53.78
auc        61.87  65.57  62.02  66.14  67.75  64.73  65.35 

SK하이닉스
              dt    gnb    knn    log     rf    svm    xgb
accuracy   56.46  59.86  62.59  59.18  59.18  65.31  56.46
precision  61.90  59.62  65.38  58.41  70.21  63.73  65.31
recall     49.37  78.48  64.56  83.54  41.77  82.28  40.51
f1         54.93  67.76  64.97  68.75  52.38  71.82  50.00
auc        57.04  70.59  63.75  69.04  63.45  65.95  65.76 

현대차
              dt    gnb    knn    log     rf    svm    xgb
accuracy   56.46  64.63  58.50  64.63  57.82  61.22  65.31
precision  52.81  66.67  54.44  62.69  53.93  56.52  58.82
recall     68.12  49.28  71.01  60.87  69.57  75.36  86.96
f1         59.49  56.67  61.64  61.7